In [59]:
import gc
import pandas as pd
import chardet
import csv

pd.options.display.float_format = '{:.10f}'.format

In [2]:
base_path = '/app/data/Desarrollos Norman/'

In [4]:
def detect_encoding(csv_path, n_bytes=100_000):
    with open(csv_path, "rb") as f:
        raw = f.read(n_bytes)
    result = chardet.detect(raw)
    return result

In [5]:
info = detect_encoding(base_path + 'SECOP_II_-_Contratos_Electrónicos_20260204.csv')
print(info)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [3]:
zonificacion_pedagogico = pd.read_excel(
    base_path + '20260212_Plantilla_Matriz_de_Contratacion.xlsx', 
    sheet_name='ZONIFICACIÓN- PEDAGOGICO',
    header=2
)

resumen_oper_directa = pd.read_excel(
    base_path + '20260212_Plantilla_Matriz_de_Contratacion.xlsx', 
    sheet_name = 'RESUMEN- OPER DIRECTA'
)

consolida_ranking = pd.read_excel(
    '/app/data/cruces SECOP/Consolida_Ranking.xlsx',
    sheet_name='Matriz_Ranking'
)

# metaverso = pd.read_excel(
#     '/app/data/Metaverso 2026.xlsx',
#     sheet_name='ZonificacionPais'
# )

# secopii_bk = pd.read_excel(
#     base_path + 'Contratos_ICBF_SECOP_II_20260204V2.xlsx'
# )

In [4]:
zonificacion_pedagogico_plus_resumen_oper_directa = pd.concat([zonificacion_pedagogico, resumen_oper_directa])

cols = [col for col in zonificacion_pedagogico_plus_resumen_oper_directa.columns if col in zonificacion_pedagogico.columns]
zonificacion_pedagogico_plus_resumen_oper_directa = zonificacion_pedagogico_plus_resumen_oper_directa[cols]

zonificacion_pedagogico_plus_resumen_oper_directa['Total_Zona'] = zonificacion_pedagogico_plus_resumen_oper_directa.groupby('ZONA 2026.')['Costo total '].transform('sum')
zonificacion_pedagogico_plus_resumen_oper_directa.loc[zonificacion_pedagogico_plus_resumen_oper_directa.duplicated('ZONA 2026.'), 'Total_Zona'] = 0

print('zonificacion_pedagogico:', zonificacion_pedagogico.shape)
print('resumen_oper_directa:', resumen_oper_directa.shape)
print('zonificacion_pedagogico_plus_resumen_oper_directa:', zonificacion_pedagogico_plus_resumen_oper_directa.shape)

zonificacion_pedagogico: (25992, 32)
resumen_oper_directa: (1672, 28)
zonificacion_pedagogico_plus_resumen_oper_directa: (27664, 32)


In [5]:
consolida_ranking = consolida_ranking[['NUMERO DE CONTRATO', 'NIT OPERADOR', 'CONSECUTIVO MATRIZ CONTRATACION']]
consolida_ranking.drop_duplicates(inplace=True)

In [6]:
concat_plus_ranking = pd.merge(
    left=zonificacion_pedagogico_plus_resumen_oper_directa,
    right=consolida_ranking,
    how='outer',
    left_on=['ZONA 2026.', 'NIT CONTRATISTA 2026'],
    right_on=['CONSECUTIVO MATRIZ CONTRATACION', 'NIT OPERADOR'],
    indicator=True
)

In [7]:
concat_plus_ranking['_merge'].value_counts()

_merge
left_only     20989
both           6733
right_only     1491
Name: count, dtype: int64

In [8]:
concat_plus_ranking.rename(columns={'_merge': 'resultado_cruce_concat_ranking'}, inplace=True)

concat_plus_ranking['resultado_cruce_concat_ranking'] = concat_plus_ranking['resultado_cruce_concat_ranking'].astype(str)

concat_plus_ranking.loc[concat_plus_ranking['resultado_cruce_concat_ranking'] == 'left_only', 'resultado_cruce_concat_ranking'] = 'Fallo: Presente solo en concat'
concat_plus_ranking.loc[concat_plus_ranking['resultado_cruce_concat_ranking'] == 'both', 'resultado_cruce_concat_ranking'] = 'Exitoso'
concat_plus_ranking.loc[concat_plus_ranking['resultado_cruce_concat_ranking'] == 'right_only', 'resultado_cruce_concat_ranking'] = 'Fallo: Presente solo en ranking'

concat_plus_ranking['resultado_cruce_concat_ranking'].value_counts()

resultado_cruce_concat_ranking
Fallo: Presente solo en concat     20989
Exitoso                             6733
Fallo: Presente solo en ranking     1491
Name: count, dtype: int64

In [9]:
print('shape', concat_plus_ranking.shape)
print('columns', concat_plus_ranking.columns)

shape (29213, 36)
columns Index(['ZONA 2026.', 'Regional UDS', 'Municipio UDS', 'Centro Zonal UDS',
       'Codigo Unidad Servicio UDS', 'Unidad Servicio UDS', 'SERVICIO 2026',
       'SERVICIO SIM', 'Componente para la UDS', 'Cupos', 'Total',
       'CONCEPTO DEFINITIVO', 'CONTRATISTA 2026', 'NIT CONTRATISTA 2026',
       'OBSERVACION', 'Conteo_Zona', 'Total_Zona', 'Meses', 'Dias',
       'Dias_Descuento', 'Alistamiento', 'Canasta Mes', 'Variables',
       'Alimentación', 'Dotación', 'Costo total ', 'Apalancamiento_2025',
       'Forma de Contratación', 'NIT CONTRATISTA 20262', 'CONTRATISTA 20263',
       'CTRL FORMA', 'JUNIO', 'NUMERO DE CONTRATO', 'NIT OPERADOR',
       'CONSECUTIVO MATRIZ CONTRATACION', 'resultado_cruce_concat_ranking'],
      dtype='object')


In [186]:
costeo_antioquia = pd.read_excel(
    '/app/data/Johan19022026/antioquia/COSTEO VF 2025-2026 152 CONT.xlsx',
    sheet_name='COSTEO',
    header=1
)

In [187]:
cols = ['ZONA 2026', 'ZONA 2026', 'N° CONTRATO', 'CENTRO ZONAL', 'MUNICIPIO', 'COMPONENTE PARA LA UDS', 'SERVICIO 2026', 'CODIGO UDS', 'UNIDAD DE SERVICIO', 'UDS AGRUPADO', 
'ATENCIONES (CUPOS)', 'ATENCIONES AGRUPADO', 'AGRUPACION', 'FORMA DE CONTRATACION', 'NIT EAS', 'EAS', 'VALOR 2025', 'VALOR 2026', 'APORTES ICBF', '% CONTRAPARTIDA',
'VALOR CONTRAPARTIDA', 'VALOR TOTAL CONTRATO AGRUPADO',  'URL SECOP']

In [189]:
costeo_antioquia = costeo_antioquia[cols]

concat_plus_ranking_plus_costeo_antioquia = pd.merge(
    left=concat_plus_ranking,
    right=costeo_antioquia,
    how='left',
    left_on=['Codigo Unidad Servicio UDS', 'Componente para la UDS'],
    right_on=['CODIGO UDS', 'COMPONENTE PARA LA UDS'],
    indicator=True
)

concat_plus_ranking_plus_costeo_antioquia['_merge'].value_counts()

_merge
left_only     27096
both           2117
right_only        0
Name: count, dtype: int64

In [190]:
concat_plus_ranking_plus_costeo_antioquia.rename(columns={'_merge': 'resultado_cruce_costeo_antioquia'}, inplace=True)

concat_plus_ranking_plus_costeo_antioquia['resultado_cruce_costeo_antioquia'] = concat_plus_ranking_plus_costeo_antioquia['resultado_cruce_costeo_antioquia'].astype(str)

concat_plus_ranking_plus_costeo_antioquia.loc[concat_plus_ranking_plus_costeo_antioquia['resultado_cruce_costeo_antioquia'] == 'left_only', 'resultado_cruce_costeo_antioquia'] = 'Fallo: Presente solo en data original'
concat_plus_ranking_plus_costeo_antioquia.loc[concat_plus_ranking_plus_costeo_antioquia['resultado_cruce_costeo_antioquia'] == 'both', 'resultado_cresultado_cruce_costeo_antioquiaruce_secop'] = 'Exitoso'

concat_plus_ranking_plus_costeo_antioquia['resultado_cruce_costeo_antioquia'].value_counts()

/tmp/ipykernel_1599/2040247071.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Exitoso' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  concat_plus_ranking_plus_costeo_antioquia.loc[concat_plus_ranking_plus_costeo_antioquia['resultado_cruce_costeo_antioquia'] == 'both', 'resultado_cresultado_cruce_costeo_antioquiaruce_secop'] = 'Exitoso'


resultado_cruce_costeo_antioquia
Fallo: Presente solo en data original    27096
both                                      2117
Name: count, dtype: int64

In [10]:
# Se usa porque el secop no está completo en DA hoy 16-02-26
secopii_bk = pd.read_excel(
    '/app/data/Contratos_ICBF_SECOP_II_20260217.xlsx'
)

In [191]:
concat_plus_ranking_plus_costeo_antioquia['NUMERO DE CONTRATO'] = concat_plus_ranking_plus_costeo_antioquia['NUMERO DE CONTRATO'].astype(str)
concat_plus_ranking_plus_costeo_antioquia['N° CONTRATO'] = concat_plus_ranking_plus_costeo_antioquia['N° CONTRATO'].astype(str)
secopii_bk['Referencia del Contrato'] = secopii_bk['Referencia del Contrato'].astype(str)

In [199]:
cruce_secop = pd.merge(
    left=concat_plus_ranking_plus_costeo_antioquia,
    right=secopii_bk,
    how='left',
    left_on=['NUMERO DE CONTRATO'],
    right_on=['Referencia del Contrato'],
    indicator=True
)

cruce_secop_no_exitoso = cruce_secop[cruce_secop['_merge'] == 'left_only']
cruce_secop_no_exitoso.drop(columns=secopii_bk.columns, inplace=True)
cruce_secop = cruce_secop[cruce_secop['_merge'] != 'left_only']
cruce_secop_no_exitoso.drop(columns=['_merge'], inplace=True)

cruce2_secop = pd.merge(
    left=cruce_secop_no_exitoso,
    right=secopii_bk,
    how='left',
    left_on=['N° CONTRATO'],
    right_on=['Referencia del Contrato'],
    indicator=True
)

cruce_secop = pd.concat([cruce_secop, cruce2_secop])
cruce_secop['_merge'].value_counts()

/tmp/ipykernel_1599/3234186623.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_secop_no_exitoso.drop(columns=secopii_bk.columns, inplace=True)


_merge
left_only     23568
both           6013
right_only        0
Name: count, dtype: int64

In [200]:
cruce_secop['_merge'].value_counts()

_merge
left_only     23568
both           6013
right_only        0
Name: count, dtype: int64

In [201]:
cruce_secop.rename(columns={'_merge': 'resultado_cruce_secop'}, inplace=True)

cruce_secop['resultado_cruce_secop'] = cruce_secop['resultado_cruce_secop'].astype(str)

cruce_secop.loc[cruce_secop['resultado_cruce_secop'] == 'left_only', 'resultado_cruce_secop'] = 'Fallo: Presente solo en data original'
cruce_secop.loc[cruce_secop['resultado_cruce_secop'] == 'both', 'resultado_cruce_secop'] = 'Exitoso'

cruce_secop['resultado_cruce_secop'].value_counts()

resultado_cruce_secop
Fallo: Presente solo en data original    23568
Exitoso                                   6013
Name: count, dtype: int64

In [202]:
cruce_secop[cruce_secop['Regional UDS'] == 'Antioquia']['resultado_cruce_secop'].value_counts()

resultado_cruce_secop
Fallo: Presente solo en data original    2283
Exitoso                                    58
Name: count, dtype: int64

In [203]:
cruce_secop_exitoso = cruce_secop[cruce_secop['resultado_cruce_secop'] == 'Exitoso']
cruce_secop_fallo = cruce_secop[cruce_secop['resultado_cruce_secop'] != 'Exitoso']

print('cruce_secop_exitoso', cruce_secop_exitoso.shape)
print('cruce_secop_fallo', cruce_secop_fallo.shape)

cruce_secop_exitoso (6013, 151)
cruce_secop_fallo (23568, 151)


In [204]:
cruce_secop.to_excel('/app/data/cruce_secop.xlsx', index=False)

In [ ]:
# def get_order_identifiers_as_list(order_identificators_list: list, chunk_size: int = 1000):
#     def clean(elem):
#         return elem.replace("\n", "").replace("\r", "").strip()

#     chunks = [
#         order_identificators_list[i:i + chunk_size]
#         for i in range(0, len(order_identificators_list), chunk_size)
#     ]

#     return [
#         "(" + ", ".join([f'"{clean(str(elem))}"' for elem in chunk]) + ")"
#         for chunk in chunks
#     ]

In [ ]:
# from typing import List, Tuple
# import numpy as np

# def money_to_float(s: pd.Series) -> pd.Series:
#     # 1) a string
#     s = s.astype(str)

#     # 2) quitar saltos de línea y espacios raros
#     s = s.str.replace(r"[\n\r\t]", "", regex=True).str.strip()

#     # 3) quitar símbolo $, espacios y separadores de miles
#     #    (si el decimal fuera coma, lo tratamos abajo)
#     s = (s
#          .str.replace("$", "", regex=False)
#          .str.replace(" ", "", regex=False)
#          .str.replace(",", "", regex=False)
#     )

#     # 4) convertir a numérico (lo que no se pueda -> NaN)
#     return pd.to_numeric(s, errors="coerce")

# def fuzzy_merge(merge_column_pairs: List[List[str]], pct: float, matriz: pd.DataFrame, secop: pd.DataFrame) -> pd.DataFrame:
#     # merge_column_pairs: [valor_l, valor_r, texto_l, texto_r]]
#     merge_result = []
#     for pair in merge_column_pairs:
#         valor_l = pair[0]; valor_r = pair[1]
#         texto_l = pair[2]; texto_r = pair[3]
        
#         # --- 0) ids para controlar 1-1 ---
#         L = matriz.copy()
#         R = secop.copy()

#         L[valor_l] = money_to_float(L[valor_l])
#         R[valor_r] = money_to_float(R[valor_r])

#         L[valor_l] = L[valor_l].astype(float)
#         R[valor_r] = R[valor_r].astype(float)

#         L["_id_l"] = np.arange(len(L))
#         R["_id_r"] = np.arange(len(R))

#         # --- 1) números ---
#         L[valor_l] = pd.to_numeric(L[valor_l], errors="coerce")
#         R[valor_r] = pd.to_numeric(R[valor_r], errors="coerce")

#         L = L.dropna(subset=[valor_l]).sort_values(valor_l).copy()
#         R = R.dropna(subset=[valor_r]).sort_values(valor_r).copy()

#         # --- 2) candidato por cercanía ---
#         cand = pd.merge_asof(
#             L,
#             R,
#             left_on=valor_l,
#             right_on=valor_r,
#             direction="nearest"
#         )

#         # --- 3) texto: normaliza a dígitos (NIT/documento) ---
#         def only_digits(x):
#             if pd.isna(x):
#                 return ""
#             return "".join(ch for ch in str(x) if ch.isdigit())

#         l_doc = cand[texto_l].map(only_digits)
#         r_doc = cand[texto_r].map(only_digits)

#         txt_ok = (l_doc == r_doc) & (r_doc != "")

#         # --- 4) tolerancia porcentual ---
#         den = cand[valor_l].abs()
#         diff = (cand[valor_l] - cand[valor_r]).abs()
#         tol_ok = np.where(den.eq(0), diff.eq(0), diff <= (pct * den))

#         cand = cand[txt_ok & tol_ok].copy()

#         # --- 5) score: menor diferencia porcentual = mejor ---
#         cand["diff_abs"] = diff[txt_ok & tol_ok]
#         cand["diff_pct"] = np.where(den[txt_ok & tol_ok].eq(0), 0, cand["diff_abs"] / den[txt_ok & tol_ok])

#         # --- 6) resolver colisiones para que sea 1-1 ---
#         # Paso A: cada L se queda con su mejor R
#         cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

#         # Paso B: si varios L apuntan al mismo R, nos quedamos con el mejor (y descartamos el resto)
#         cand = cand.sort_values(["_id_r", "diff_pct"]).drop_duplicates("_id_r", keep="first")

#         # (Opcional) Paso C: volver a asegurar unicidad por L (por si el paso B dejó huecos)
#         cand = cand.sort_values(["_id_l", "diff_pct"]).drop_duplicates("_id_l", keep="first")

#         out = cand.drop(columns=["diff_abs", "diff_pct", "_id_r", "_id_l"])

#         merge_result.append(out)
    
#     return pd.concat(merge_result)

In [ ]:
# pct = 0.8  # 3% tolerancia

# merge_column_pairs = [
#     ['Total_Zona', 'Valor del Contrato', 'NIT OPERADOR', 'Documento Proveedor'],   
#     ['Total_Zona', 'Valor del Contrato', 'NIT OPERADOR', 'Identificación Representante Legal'],
# ]

# cruce_secop_fallo['Total_Zona'] = cruce_secop_fallo['Total_Zona'].astype(str)
# cruce_secop_fallo['NIT OPERADOR'] = cruce_secop_fallo['NIT OPERADOR'].astype(str)
# secopii_bk['Documento Proveedor'] = secopii_bk['Documento Proveedor'].astype(str)
# secopii_bk['Identificación Representante Legal'] = secopii_bk['Identificación Representante Legal'].astype(str)
# secopii_bk['Valor del Contrato'] = secopii_bk['Valor del Contrato'].astype(str)

# merge = fuzzy_merge(merge_column_pairs, pct, cruce_secop_fallo, secopii_bk)